In [59]:
!pip install -r requirements.txt

In [60]:
from dotenv import load_dotenv

load_dotenv()

True

In [61]:
from langchain.llms import OpenAI

In [62]:
import os

openai_api_key = os.environ.get('OPENAI_API_KEY')

In [63]:
llm = OpenAI(model_name="text-ada-001", n=2, best_of=2)

In [64]:
llm("Tell me a joke")

'\n\nWhy did the chicken cross the road?\n\nTo get to the other side!'

In [65]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [66]:
chat = ChatOpenAI(temperature=0)

In [67]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I love programming.")
]

In [68]:
chat(messages)

AIMessage(content="J'adore la programmation.", additional_kwargs={})

In [69]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love artificial intelligence.")
    ],
]

In [71]:
result = chat.generate(batch_messages)

In [ ]:
result

LLMResult(generations=[[ChatGeneration(text="J'adore la programmation.", generation_info=None, message=AIMessage(content="J'adore la programmation.", additional_kwargs={}))], [ChatGeneration(text="J'adore l'intelligence artificielle.", generation_info=None, message=AIMessage(content="J'adore l'intelligence artificielle.", additional_kwargs={}))]], llm_output={'token_usage': {'prompt_tokens': 57, 'completion_tokens': 20, 'total_tokens': 77}, 'model_name': 'gpt-3.5-turbo'})

In [ ]:
result.llm_output

{'token_usage': {'prompt_tokens': 57,
  'completion_tokens': 20,
  'total_tokens': 77},
 'model_name': 'gpt-3.5-turbo'}

In [72]:
gpt4 = ChatOpenAI(model="gpt-4", temperature=0)

In [ ]:
gpt4(messages)

AIMessage(content="J'aime la programmation.", additional_kwargs={})

In [ ]:
def chatmongo(query,database):
    system = SystemMessage(content="You are a MongoDB query generator for a database with the following collections and their properties:\n\n\n" + database )
    user = HumanMessage(content=query)
    x = chat([system,user])
    
    return x 

In [ ]:
import gradio as gr

In [ ]:
examples = [
    ["List the names of the departments which employed more than 10 employees","Employee(_id, name, department_id)\nDepartment(_id, name, address)\nSalary_Payments(_id, employee_id, amount, date)"],
    ["List the names of the departments which employed less than 10 employees","Employee(_id, name, department_id)\nDepartment(_id, name, address)\nSalary_Payments(_id, employee_id, amount, date)",]
] 

In [ ]:
example = chatmongo("List the names of the departments which employed more than 10 employees","Employee(_id, name, department_id)\nDepartment(_id, name, address)\nSalary_Payments(_id, employee_id, amount, date)")
example

AIMessage(content='To list the names of the departments which employed more than 10 employees, we need to perform a join between the Employee and Department collections, group the result by department_id and count the number of employees in each department. Then, we can filter the departments with more than 10 employees and project only the department names. The MongoDB query for this would be:\n\n```\ndb.Employee.aggregate([\n  {\n    $lookup: {\n      from: "Department",\n      localField: "department_id",\n      foreignField: "_id",\n      as: "department"\n    }\n  },\n  {\n    $unwind: "$department"\n  },\n  {\n    $group: {\n      _id: "$department._id",\n      name: { $first: "$department.name" },\n      count: { $sum: 1 }\n    }\n  },\n  {\n    $match: {\n      count: { $gt: 10 }\n    }\n  },\n  {\n    $project: {\n      _id: 0,\n      name: 1\n    }\n  }\n])\n```\n\nThis query first performs a `$lookup` to join the Employee and Department collections based on the `department_i

In [77]:
def chat_fewshot_mongo(query,database):
    mongo_messages = []
    system = SystemMessage(content="You are a MongoDB query generator for a database with the following collections and their properties:\n\n\n" + database )
    user_1 = HumanMessage(content="List the names of the departments which employed more than 10 employees in the last 3 months.")
    ai_1 = AIMessage(content="db.Department.aggregate([\n  {\n    $lookup: {\n      from: \"Employee\",\n      localField: \"_id\",\n      foreignField: \"department_id\",\n      as: \"employees\"\n    }\n  },\n  {\n    $lookup: {\n      from: \"Salary_Payments\",\n      localField: \"employees._id\",\n      foreignField: \"employee_id\",\n      as: \"salary_payments\"\n    }\n  },\n  {\n    $match: {\n      \"salary_payments.date\": { $gte: new Date(new Date().getTime() - 1000 * 60 * 60 * 24 * 90) }\n    }\n  },\n  {\n    $group: {\n      _id: \"$name\",\n      employeeCount: { $sum: 1 }\n    }\n  },\n  {\n    $match: {\n      employeeCount: { $gt: 10 }\n    }\n  },\n  {\n    $project: {\n      _id: 0,\n      department_name: \"$_id\"\n    }\n  }\n]);\n'''\n" )
    user_2 = HumanMessage(content="List the names of the departments which employed more than 3 employees and has id less than 3400." )
    ai_2 = AIMessage(content="db.Department.aggregate([\n  {\n    $match: { _id: { $lt: 3400 } }\n  },\n  {\n    $lookup: {\n      from: \"Employee\",\n      localField: \"_id\",\n      foreignField: \"department_id\",\n      as: \"employees\"\n    }\n  },\n  {\n    $group: {\n      _id: \"$name\",\n      employeeCount: { $sum: { $size: \"$employees\" } }\n    }\n  },\n  {\n    $match: {\n      employeeCount: { $gt: 3 }\n    }\n  },\n  {\n    $project: {\n      _id: 0,\n      department_name: \"$_id\"\n    }\n  }\n]);\n'''")
    user = HumanMessage(content=query)
    mongo_messages.append(system)
    mongo_messages.append(user_1)
    mongo_messages.append(ai_1)
    mongo_messages.append(user_2)
    mongo_messages.append(ai_2)
    mongo_messages.append(user)

    x = chat(mongo_messages)
    
    return x 

In [73]:
example1 = chat_fewshot_mongo("List the names of the departments which employed more than 10 employees","Employee(_id, name, department_id)\nDepartment(_id, name, address)\nSalary_Payments(_id, employee_id, amount, date)")
example1

AIMessage(content='db.Department.aggregate([\n  {\n    $lookup: {\n      from: "Employee",\n      localField: "_id",\n      foreignField: "department_id",\n      as: "employees"\n    }\n  },\n  {\n    $group: {\n      _id: "$name",\n      employeeCount: { $sum: { $size: "$employees" } }\n    }\n  },\n  {\n    $match: {\n      employeeCount: { $gt: 10 }\n    }\n  },\n  {\n    $project: {\n      _id: 0,\n      department_name: "$_id"\n    }\n  }\n]);\n\'\'\'', additional_kwargs={})

In [74]:
with gr.Blocks() as demo:
    textbox = gr.Textbox(placeholder="Type here...", lines=10)
    btn = gr.Button("Complete")
    
    # define what will run when the button is clicked, here the textbox is used as both an input and an output
    btn.click(fn=chat_fewshot_mongo, inputs=textbox, outputs=textbox, queue=False)

/home/orientusprime/miniconda3/envs/textgen/lib/python3.10/site-packages/gradio/utils.py:851: UserWarning: Expected 2 arguments for function <function chat_fewshot_mongo at 0x7fd938a06170>, received 1.
  warnings.warn(
/home/orientusprime/miniconda3/envs/textgen/lib/python3.10/site-packages/gradio/utils.py:855: UserWarning: Expected at least 2 arguments for function <function chat_fewshot_mongo at 0x7fd938a06170>, received 1.
  warnings.warn(


In [75]:
demo = gr.Interface(
    fn=chat_fewshot_mongo,
    inputs=[
        gr.components.Textbox(lines=5, label="Natural Language Query"),
        gr.components.Textbox(lines=5, label="Database Format"),
    ],
    outputs=gr.components.Textbox(label="NoSQL Output"),
    examples=examples
) 

In [76]:
demo.launch(share=False, debug=True) 

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
